In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.authentication import InteractiveLoginAuthentication

# Force an interactive login to ensure you're authenticated under the correct account.
auth = InteractiveLoginAuthentication(force=True)
ws = Workspace.from_config(auth=auth)

from azureml.core import Experiment

# If this call fails with a Forbidden error, it likely means you don't have the correct permissions.
# Get more details about the workspace
details = ws.get_details()
print(details)
all_experiments = Experiment.list(ws)
for e in all_experiments:
    print(e.name)
exp = Experiment(ws, "test-auto")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')



run = exp.start_logging()

{'id': '/subscriptions/509b19dc-1a93-481c-afae-84a0e77eabd7/resourceGroups/AI-Class/providers/Microsoft.MachineLearningServices/workspaces/class', 'name': 'class', 'identity': {'principal_id': 'f682d936-cd0f-4346-a56d-b7bc2619958c', 'tenant_id': '21c9bdcd-4cda-47b9-80cb-dcc713e91ce3', 'type': 'SystemAssigned'}, 'location': 'eastus2', 'type': 'Microsoft.MachineLearningServices/workspaces', 'tags': {}, 'sku': 'Basic', 'workspaceid': '44d44120-85a3-42ba-b806-eb67db8c1674', 'sdkTelemetryAppInsightsKey': 'e1f7b545-6243-4abf-ba76-c5691d2edb62', 'description': '', 'friendlyName': 'class', 'creationTime': '2024-11-11T20:08:41.7479622Z', 'keyVault': '/subscriptions/509b19dc-1a93-481c-afae-84a0e77eabd7/resourceGroups/AI-Class/providers/Microsoft.Keyvault/vaults/class0232282594', 'applicationInsights': '/subscriptions/509b19dc-1a93-481c-afae-84a0e77eabd7/resourceGroups/AI-Class/providers/Microsoft.insights/components/class1211978839', 'storageAccount': '/subscriptions/509b19dc-1a93-481c-afae-84a0

In [39]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "classcluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException


# Check if the compute target already exists
try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing cluster '{cluster_name}'. Using it.")
except ComputeTargetException:
    print(f"Creating a new cluster '{cluster_name}'.")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_D2_V2",  # Use the specified VM size
        max_nodes=4               # No more than 4 nodes
    )
    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_cluster.wait_for_completion(show_output=True)

print("Cluster is ready for use.")

Found existing cluster 'classcluster'. Using it.
Cluster is ready for use.


In [10]:
#from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.1, 1.0),        # Hyperparameter "C" as a float uniform range
        "--max_iter": choice(50, 100, 150)  # Choose among discrete integer values
    }
)

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval=2,  # Frequency for applying policy
    slack_factor=0.1        # Allowed slack ratio
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            compute_target=compute_cluster,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",          # The metric you want to optimize
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,                      # Total hyperparameter runs
    max_concurrent_runs=4                   # How many runs to execute in parallel
)

In [5]:
%pip install azureml-widgets

Note: you may need to restart the kernel to use updated packages.


In [6]:
from azureml.widgets import RunDetails

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)


RunDetails(hyperdrive_run).show()



_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [12]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_9ed9b4e4-e308-4afe-ba08-91c18de02537
Web View: https://ml.azure.com/runs/HD_9ed9b4e4-e308-4afe-ba08-91c18de02537?wsid=/subscriptions/509b19dc-1a93-481c-afae-84a0e77eabd7/resourcegroups/AI-Class/workspaces/class&tid=21c9bdcd-4cda-47b9-80cb-dcc713e91ce3

Streaming azureml-logs/hyperdrive.txt

[2024-12-15T02:53:27.8873302Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2024-12-15T02:53:28.1435383Z][SCHEDULER][INFO]Scheduling job, id='HD_9ed9b4e4-e308-4afe-ba08-91c18de02537_0' 
[2024-12-15T02:53:28.2482682Z][SCHEDULER][INFO]Scheduling job, id='HD_9ed9b4e4-e308-4afe-ba08-91c18de02537_3' 
[2024-12-15T02:53:28.2495002Z][SCHEDULER][INFO]Scheduling job, id='HD_9ed9b4e4-e308-4afe-ba08-91c18de02537_1' 
[2024-12-15T02:53:28.2504571Z][SCHEDULER][INFO]Scheduling job, id='HD_9ed9b4e4-e308-4afe-ba08-91c18de02537_2' 
[2024-12-15T02:53:28.6733427Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_9ed9b4e4-e308-4afe-ba08-91c18de02537_2' 
[2024-12-15T02:53:28.7133266Z][SCHEDULER][INFO]

{'runId': 'HD_9ed9b4e4-e308-4afe-ba08-91c18de02537',
 'target': 'classcluster',
 'status': 'Completed',
 'startTimeUtc': '2024-12-15T02:53:26.162546Z',
 'endTimeUtc': '2024-12-15T03:12:37.398446Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4d98e09e-fb99-451b-a7ac-43bd58f07e3d',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_9ed9b4e4-e308-4afe-ba08-91c18de02537_14',
  'score': '0.9089529590288316',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_9ed9b4e4-e308-4afe-ba08-91c18de02537_14'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sd

In [17]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_run_metrics = best_run.get_metrics()
print("Best Run Metrics:", best_run_metrics)



Best Run Metrics: {'Regularization Strength:': 0.24948536814716105, 'Max iterations:': 100, 'Accuracy': 0.9089529590288316}


In [38]:
# Get the output directory of the best run and load the model file
best_run_metrics = best_run.get_metrics()
print("Best Run Metrics:", best_run_metrics)

Best Run Metrics: {'Regularization Strength:': 0.24948536814716105, 'Max iterations:': 100, 'Accuracy': 0.9089529590288316}


In [18]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

datastore_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
factory = TabularDatasetFactory()
ds = factory.from_delimited_files(path=datastore_path)
print(ds.to_pandas_dataframe())

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
       age          job  marital          education  default housing loan  \
0       57   technician  married        high.school       no      no  yes   
1       55      unknown  married            unknown  unknown     yes   no   
2       33  blue-collar  married           basic.9y       no      no   no   
3       36       admin.  married        high.school       no      no   no   
4       27    housemaid  married        high.school       no     yes   no   
...    ...          ...      ...                ...      ...     ...  ...   
32945   56    housemaid  married           basic.4y       no      no  yes   
32946   37   management  married  university.degree       no      no  yes   
32947   26       admin.   single  university.degree       no      no   no   
32948   31  blue-collar   single           basic.9y       no   

In [19]:
from train import clean_data
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y)

AMLdata = x_train.join(y_train)
AMLdata.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
       age  marital  default  housing  loan  month  day_of_week  duration  \
0       57        1        0        0     1      5            1       371   
1       55        1        0        1     0      5            4       285   
2       33        1        0        0     0      5            5        52   
3       36        1        0        0     0      6            5       355   
4       27        1        0        1     0      7            5       189   
...    ...      ...      ...      ...   ...    ...          ...       ...   
32945   56        1        0        0     1      7            1       116   
32946   37        1        0        0     1      7            5        69   
32947   26        0        0        0     0      5            2       135   
32948   31        0        0        0     0      4            1

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
9720,31,0,0,1,0,11,4,228,1,999,...,0,0,0,0,0,0,0,1,0,0
30089,39,0,0,0,0,5,4,122,2,999,...,0,0,0,0,0,0,0,1,0,0
27005,55,0,0,0,1,5,3,217,2,999,...,1,0,0,0,0,0,1,0,0,0
82,46,1,0,1,1,5,2,409,2,999,...,1,1,0,0,0,0,0,0,0,0
18899,42,1,0,0,0,4,5,46,4,999,...,0,0,0,0,0,0,0,1,0,0


In [24]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='accuracy',
    X=x_train,
    y=y_train,
    n_cross_validations=3)

In [25]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config)
automl_run.wait_for_completion(show_output = True)

Experiment,Id,Type,Status,Details Page,Docs Page
test-auto,AutoML_2f18504f-b2c2-4934-8f26-8a416f93fd97,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2024/12/15 03:23:13 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Experiment,Id,Type,Status,Details Page,Docs Page
test-auto,AutoML_2f18504f-b2c2-4934-8f26-8a416f93fd97,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2749                          |1                               |24712                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_2f18504f-b2c2-4934-8f26-8a416f93fd97',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2024-12-15T03:22:37.085072Z',
 'endTimeUtc': '2024-12-15T03:52:29.889378Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"test-auto","subscription_id":"509b19dc-1a93-481c-afae-84a0e77eabd7","resource_group":"AI-Class","workspace_name":"class","region":"eastus2","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM

In [26]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, fitted_automl_model = automl_run.get_output()
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
test-auto,AutoML_2f18504f-b2c2-4934-8f26-8a416f93fd97_35,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [32]:
print("Accuracy:", best_automl_run.get_metrics(name="accuracy"))
print()
print(fitted_automl_model.steps[-1])

Accuracy: {'accuracy': 0.9170848315583743}

('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('30', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=0.6, colsample_bytree=1, eta=0.001, gamma=0, max_depth=6, max_leaves=15, n_estimators=800, n_jobs=1, objective='reg:logis

In [31]:
best_automl_run.register_model(model_name='AutoML_best.pkl', model_path='./outputs/')


Model(workspace=Workspace.create(name='class', subscription_id='509b19dc-1a93-481c-afae-84a0e77eabd7', resource_group='AI-Class'), name=AutoML_best.pkl, id=AutoML_best.pkl:1, version=1, tags={}, properties={})

In [41]:
compute_cluster.delete()